In [32]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

- 검색어 space replace

In [2]:
book_info_total = pd.read_pickle('./book_info_total.pkl')
book_info_total

,title,author,rating,category,tag,keyword,image,title_concat,feature
0,매매의 기술,박병창,9.5,주식 투자 재테크 재테크일반 경제 경영 증권,주식투자입문 주식투자,,http://image.yes24.com/goods/101666872/L,매매의기술,박병창 주식 투자 재테크 재테크일반 경제 경영 증권 주식투자입문 주식투자
1,부의 시나리오,오건영,NaN,경제 경영 투자 재테크 재테크일반,투자트렌드 포트폴리오 삼프로TV,금융시장 투자 경제 오건,http://image.yes24.com/goods/101874075/L,부의시나리오,오건영 경제 경영 투자 재테크 재테크일반 투자트렌드 포트폴리오 삼프로TV 금융시...
2,라오어의 미국주식 무한매수법,라오어,10.0,주식 투자 재테크 경제 경영 증권,미국주식,,http://image.yes24.com/goods/101875887/L,라오어의미국주식무한매수법,라오어 주식 투자 재테크 경제 경영 증권 미국주식
3,부자 아빠 가난한 아빠 20주년 특별 기념판,로버트 기요사키 안진환,9.4,경제 경영 투자 재테크 재테크일반,돈모으고싶을때 금융문맹탈출 가난이싫어 부자되는법,아빠 금융 부자 세계 투자 아버 대한 가지 사람 내용,http://image.yes24.com/goods/58774995/L,부자아빠가난한아빠20주년특별기념판,로버트 기요사키 안진환 경제 경영 투자 재테크 재테크일반 돈모으고싶을때 금융문맹...
4,나는 당신이 주식 공부를 시작했으면 좋겠습니다,이효석,10.0,주식 투자 재테크 경제 경영 증권,삼프로TV 주식투자입문 주식투자,투자 생존 보이지 주식 저자 미래 전문,http://image.yes24.com/goods/101619905/L,나는당신이주식공부를시작했으면좋겠습니다,이효석 주식 투자 재테크 경제 경영 증권 삼프로TV 주식투자입문 주식투자 투자 ...
...,...,...,...,...,...,...,...,...,...
4239,뇌를 최적화시키는 생활명상 10분,신재한 임운나,NaN,뇌과학 뇌과학 일반 자연과학,,마음 상태 의식 목적 명상,http://image.yes24.com/goods/90584042/L,뇌를최적화시키는생활명상10분,신재한 임운나 뇌과학 뇌과학 일반 자연과학 마음 상태 의식 목적 명상
4240,"아빠, 천체관측 떠나요!",조상호,9.7,천문학 자연과학 별자리와 천체관측,,,http://image.yes24.com/goods/90175509/L,"아빠,천체관측떠나요!",조상호 천문학 자연과학 별자리와 천체관측
4241,인체 의학 도감 베스트 4종 세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지,NaN,의학 재미있는 인체이야기 자연과학 인체 건강에세이 건강기타 건강 취미 약학,,,http://image.yes24.com/goods/89970464/L,인체의학도감베스트4종세트,노가미 하루오 마쓰무라 다카히로 야마모토 나오마사 야마구치 슌페이 다케우치 슈지 ...
4242,세상에서 가장 재미있는 생물학,데이브 웨스너 래리 고닉 김소정,10.0,생물학 생명과학 재미있는 생명이야기 자연과학,,생물학 가장 웨스 세상 시리 하버드 만화 데이브 과학 만점,http://image.yes24.com/goods/89907468/L,세상에서가장재미있는생물학,데이브 웨스너 래리 고닉 김소정 생물학 생명과학 재미있는 생명이야기 자연과학 생...


# 1. 책 제목을 입력으로 받아 비슷한 책 제목 띄워주기
- 띄어쓰기 해도 ok, 안해도 ok

In [55]:
# 사용자가 검색어로 넣는 책 이름의 띄어쓰기가 정확하지 않아도 검색 가능하도록 함
def get_book_idx_by_title(book_title):
    # 책 제목이 검색어와 완전히 일치하지 않아도 포함만 되면 검색 되도록 함
    book_title_concat = book_title.replace(' ', '')
    book_idx = book_info_total.index[(book_info_total['title_concat'] == book_title_concat)].to_list()
    if book_idx == []:
        book_idx = book_info_total.index[(book_info_total['title_concat'].str.contains(book_title_concat))].to_list()
    if book_idx:
        #print(len(book_idx))
        i = random.randrange(len(book_idx))
        return book_idx[i]
    else:
        return None

def get_book_title(book_idx):
    if book_idx in book_info_total.index.values:
        return book_info_total.loc[book_idx]['title']
    else:
        return None
    
# 사용자가 검색어로 넣은 단어가 책의 feature keywords 중 하나와 매치되는지
def get_book_idx_by_keyword(keyword):
    book_idx = book_info_total.index[(book_info_total['feature'].str.contains(keyword))].to_list()
    if book_idx:
        #print(len(book_idx))
        i = random.randrange(len(book_idx))
        return book_idx[i]
    else:
        return None

In [62]:
print(get_book_idx_by_keyword('책읽어드립니다'))

1400


In [42]:
# min_df를 1로 설정해줌으로써 한번이라도 노출된 정보도 다 고려함
# ngram_range: n_gram 범위 지정 연속으로 나오는 단어들의 순서도 고려함
tf = TfidfVectorizer(min_df=1, ngram_range=(1,5))
tfidf_matrix = tf.fit_transform(book_info_total['feature'])
tfidf_matrix

<4244x165244 sparse matrix of type '<class 'numpy.float64'>'
	with 286929 stored elements in Compressed Sparse Row format>

In [6]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.07425905, 0.14541622, ..., 0.        , 0.        ,
        0.        ],
       [0.07425905, 1.        , 0.04862108, ..., 0.        , 0.        ,
        0.        ],
       [0.14541622, 0.04862108, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.00415701,
        0.00476445],
       [0.        , 0.        , 0.        , ..., 0.00415701, 1.        ,
        0.00457294],
       [0.        , 0.        , 0.        , ..., 0.00476445, 0.00457294,
        1.        ]])

In [7]:
def similar_books_by_title(title, n=10):
    # 해당 제목의 책의 index를 구한다.
    book_idx = get_book_idx_by_title(title)
    if book_idx is None:
        print("해당 도서는 데이터셋에 존재하지 않습니다. 다른 도서나 키워드로 다시 검색해주세요.\n혹시 띄어쓰기를 하셨다면 띄어쓰기를 제거한 제목을 입력해주세요.")
    else:
        print(book_idx)
        # 모든 책에 대해서 해당 책과의 cosine similarity를 구한다.
        book_similarities = list(enumerate(cosine_sim[book_idx]))
        # cosine similarity에 따라 책들을 정렬한다.
        book_similarities = sorted(book_similarities, key=lambda x:x[1], reverse=True)
        most_similar_books = list(map(lambda x: (get_book_title(x[0]), x[0]), book_similarities[:n+1]))       
        most_similar_books = list(filter(lambda x: title != x[0], most_similar_books))
        print(most_similar_books)
        
# 카테고리, 저자, 키워드로 검색
def similar_books_by_keyword(keyword, n=10):
    # 해당 제목의 책의 index를 구한다.
    book_idx = get_book_idx_by_keyword(keyword)
    if book_idx is None:
        print("해당 키워드는 검색이 불가능합니다. 다른 도서나 키워드, 카테고리, 저자명으로 다시 검색해주세요.\n혹시 여러 키워드를 입력하였다면 개수를 줄여주세요.")
    else:
        print(book_idx)
        # 모든 책에 대해서 해당 책과의 cosine similarity를 구한다.
        book_similarities = list(enumerate(cosine_sim[book_idx]))
        # cosine similarity에 따라 책들을 정렬한다.
        book_similarities = sorted(book_similarities, key=lambda x:x[1], reverse=True)
        most_similar_books = list(map(lambda x: (get_book_title(x[0]), x[0]), book_similarities[:n+1]))       
        most_similar_books = list(filter(lambda x: keyword != x[0], most_similar_books))
        print(most_similar_books)

### 최종 코드

In [117]:
def similar_books_by_title(title, n=10):
    title = title.replace(' ', '')
    # 해당 제목의 책의 index를 구한다.
    book_idx = get_book_idx_by_title(title)
    if book_idx is None:
        print("해당 도서는 데이터셋에 존재하지 않습니다. 다른 도서나 키워드로 다시 검색해주세요.\n혹시 띄어쓰기를 하셨다면 띄어쓰기를 제거한 제목을 입력해주세요.")
    else:
        print(book_idx)
        # 모든 책에 대해서 해당 책과의 cosine similarity를 구한다.
        book_similarities = list(enumerate(cosine_sim[book_idx]))
        # cosine similarity에 따라 책들을 정렬한다.
        book_similarities = sorted(book_similarities, key=lambda x:x[1], reverse=True)
        most_similar_books = list(map(lambda x: (get_book_title(x[0]), x[0]), book_similarities[:n+1]))       
        most_similar_books = list(filter(lambda x: title != x[0].replace(' ',''), most_similar_books))
        print(len(most_similar_books))
        book_indices = []
        if len(most_similar_books) > n:
            for i, j in book_similarities[:n]:
                book_indices.append(i)
        else:
            for i, j in book_similarities[1:n+1]:
                book_indices.append(i) 
        print(most_similar_books, book_indices)
        return(book_indices)
        
# 카테고리, 저자, 키워드로 검색
def similar_books_by_keyword(keyword, n=10):
    # 해당 제목의 책의 index를 구한다.
    book_idx = get_book_idx_by_keyword(keyword)
    if book_idx is None:
        print("해당 키워드는 검색이 불가능합니다. 다른 도서나 키워드, 카테고리, 저자명으로 다시 검색해주세요.\n혹시 여러 키워드를 입력하였다면 개수를 줄여주세요.")
    else:
        print(book_idx)
        # 모든 책에 대해서 해당 책과의 cosine similarity를 구한다.
        book_similarities = list(enumerate(cosine_sim[book_idx]))
        # cosine similarity에 따라 책들을 정렬한다.
        book_similarities = sorted(book_similarities, key=lambda x:x[1], reverse=True)
        most_similar_books = list(map(lambda x: (get_book_title(x[0]), x[0]), book_similarities[:n]))       
        most_similar_books = list(filter(lambda x: keyword != x[0], most_similar_books))
        
        book_indices = []
        for i, j in book_similarities[:n]:
            book_indices.append(i)
        print(most_similar_books, book_indices)
        return(book_indices)

### 제목 기반 비슷한 책 추천
- 사용자가 띄어쓰기 오류를 낼 수 있으므로 띄어쓰기를 하지 않아도, 띄어쓰기를 해도 검색 가능하도록 함

In [121]:
idx_1 = similar_books_by_title('뇌를최적화시키는생활명상10분')
result_df = book_info_total.loc[idx_1,['title', 'author', 'rating', 'category', 'tag', 'keyword', 'image']]
result_df

4239
10
[('박문호 박사의 뇌과학 공부', 3910), ('그림으로 읽는 뇌과학의 모든 것', 4062), ('내 머릿속에선 무슨 일이 벌어지고 있을까', 3946), ('지능의 탄생', 3922), ('당신의 뇌, 미래의 뇌', 4050), ('NEWTON HIGHLIGHT 뉴턴 하이라이트 128 기본부터 알아보는 심리학', 4126), ('더 브레인', 3911), ('마음의 오류들 ', 3899), ('사이코패스 뇌과학자', 3056), ('슈퍼인텔리전스', 3983)] [3910, 4062, 3946, 3922, 4050, 4126, 3911, 3899, 3056, 3983]


,title,author,rating,category,tag,keyword,image
3910,박문호 박사의 뇌과학 공부,박문호,9.5,뇌과학 뇌과학 일반 자연과학,뇌가내를안다면 신경과학 뇌과학,기능 공부 구조 과학 대한,http://image.yes24.com/goods/56774669/L
4062,그림으로 읽는 뇌과학의 모든 것,박문호,9.9,뇌과학 뇌과학 일반 자연과학,뇌가내를안다면,신경 내용 그림 강의 과학,http://image.yes24.com/goods/8673961/L
3946,내 머릿속에선 무슨 일이 벌어지고 있을까,김대식,9.0,뇌과학 뇌과학 일반 자연과학,뇌가내를안다면,세상 사람 머릿 정리 과학,http://image.yes24.com/goods/13491020/L
3922,지능의 탄생,이대열,NaN,뇌과학 뇌과학 일반 자연과학,,지능 인공 대열 의미 교수 대한 시대 본질,http://image.yes24.com/goods/99481342/L
4050,"당신의 뇌, 미래의 뇌",김대식,8.9,경향신문 동아일보 자연과학 뇌과학 뇌과학 일반 미디어 추천,신경과학 뇌과학,우리,http://image.yes24.com/goods/76595783/L
4126,NEWTON HIGHLIGHT 뉴턴 하이라이트 128 기본부터 알아보는 심리학,뉴턴프레스,NaN,심리 인문 자연과학 심리학 이론 뇌과학 뇌과학 일반,,,http://image.yes24.com/goods/71929659/L
3911,더 브레인,데이비드 이글먼 전대호,9.2,경향신문 자연과학 뇌과학 뇌과학 일반 미디어 추천,뇌과학 빨간책방,방송,http://image.yes24.com/goods/43887148/L
3899,마음의 오류들,에릭 캔델 이한음,9.8,자연과학 조선일보 뇌과학 뇌과학 일반 미디어 추천,국립중앙도서관사서추천 노벨상수상자 감정공부 교양쌓기 뇌과학,,http://image.yes24.com/goods/90984668/L
3056,사이코패스 뇌과학자,제임스 팰런 김미선,9.1,심리 인문 자연과학 뇌과학 뇌과학 일반,사이코패스 뇌과학,사이코패스 사진 과학자 주인 스캔 가족,http://image.yes24.com/goods/92853837/L
3983,슈퍼인텔리전스,닉 보스트롬,10.0,경향신문 동아일보 중앙일보 자연과학 조선일보 뇌과학 뇌과학 일반 미디어 추천,빌게이츠추천,지능 개발 인공,http://image.yes24.com/goods/38292582/L


In [124]:
result_json = result_df.to_json(orient = 'records', force_ascii=False)
result_json

'[{"title":"박문호 박사의 뇌과학 공부","author":"박문호 ","rating":9.5,"category":"뇌과학 뇌과학 일반 자연과학","tag":" 뇌가내를안다면  신경과학 뇌과학","keyword":"기능 공부 구조 과학 대한 ","image":"http:\\/\\/image.yes24.com\\/goods\\/56774669\\/L"},{"title":"그림으로 읽는 뇌과학의 모든 것","author":"박문호 ","rating":9.9,"category":"뇌과학 뇌과학 일반 자연과학","tag":" 뇌가내를안다면 ","keyword":"신경 내용 그림 강의 과학 ","image":"http:\\/\\/image.yes24.com\\/goods\\/8673961\\/L"},{"title":"내 머릿속에선 무슨 일이 벌어지고 있을까","author":"김대식 ","rating":9.0,"category":"뇌과학 뇌과학 일반 자연과학","tag":" 뇌가내를안다면 ","keyword":"세상 사람 머릿 정리 과학 ","image":"http:\\/\\/image.yes24.com\\/goods\\/13491020\\/L"},{"title":"지능의 탄생","author":"이대열 ","rating":null,"category":"뇌과학 뇌과학 일반 자연과학","tag":"","keyword":"지능 인공 대열 의미 교수 대한 시대 본질 ","image":"http:\\/\\/image.yes24.com\\/goods\\/99481342\\/L"},{"title":"당신의 뇌, 미래의 뇌","author":"김대식 ","rating":8.9,"category":"경향신문 동아일보 자연과학 뇌과학 뇌과학 일반 미디어 추천","tag":" 신경과학 뇌과학","keyword":"우리 ","image":"http:\\/\\/image.yes24.com\\/goods\\/76595783\\/L"},{"title":"NEWTON HIGHLIGHT 뉴

In [111]:
similar_books_by_title('아내를모자로 착각한남자')

2944
10
[('적정한 삶 ', 2875), ('행복의 정복', 3249), ('솔직하게, 상처 주지 않게', 3239), ('가족의 두 얼굴', 3020), ('그림으로 읽는 생생 심리학', 3259), ('사람이 힘겨운 당신을 위한 관계의 심리학', 3255), ('불안한 마음을 잠재우는 법', 2973), ('이게 다 심리학 덕분이야', 3141), ('인간 관계의 법칙', 3081), ('걱정이 많은 사람을 위한 심리학 수업', 2962)] [2875, 3249, 3239, 3020, 3259, 3255, 2973, 3141, 3081, 2962]


In [88]:
book_info_total.loc[[2875, 3249, 3239, 3020, 3259,3255, 2973, 3141, 3081, 2962],['title','feature']]

,title,feature
2875,적정한 삶,김경일 쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리
3249,행복의 정복,버트런드 러셀 이순희 쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리
3239,"솔직하게, 상처 주지 않게",전미경 쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리 책읽아웃 마음다스리기 감...
3020,가족의 두 얼굴,최광현 쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리 가족 상처 아픔
3259,그림으로 읽는 생생 심리학,이소라 남종호 쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리 심리 생생 그림
3255,사람이 힘겨운 당신을 위한 관계의 심리학,최광현 쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리 심리학처방전 트라우마 관...
2973,불안한 마음을 잠재우는 법,하주원 쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리 무너지지마요 마음들여다보...
3141,이게 다 심리학 덕분이야,주현성 쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리 심리 우울 스킬 마음 일상
3081,인간 관계의 법칙,로버트 그린 강미경 쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리 인간내면탐구...
2962,걱정이 많은 사람을 위한 심리학 수업,채드 르쥔느 조영지 쉽게 읽는 심리학 인문 주제로 읽는 심리학 심리 걱정 우리 불안


In [115]:
similar_books_by_title('시간의역사')

3873
11
[('그림으로 보는 시간의 역사', 3873), ('우주를 정복하는 딱 10가지 지식', 4183), ('짧고 쉽게 쓴 시간의 역사', 4097), ('NEWTON HIGHLIGHT 뉴턴 하이라이트 136 우주 대도감', 3992), ('우주와 천체의 원리를 그림으로 쉽게 풀이한 천문학 사전', 3927), ('100개의 별, 우주를 말하다', 4107), ('우리는 모두 별에서 왔다', 4087), ('비욘드 그래비티', 4033), ('아시모프의 코스모스', 4010), ('지구인의 우주공부', 4226), ('모든 사람을 위한 빅뱅 우주론 강의', 3965)] [3873, 4183, 4097, 3992, 3927, 4107, 4087, 4033, 4010, 4226]


In [91]:
book_info_total.loc[[3873, 4183, 4097,3992, 3927, 4107, 4087, 4033, 4010, 4226, 3965], ['title', 'feature']]

,title,feature
3873,그림으로 보는 시간의 역사,스티븐 호킹 김동광 재미있는 우주이야기 천문학 자연과학 천문학 일반 20세기고전...
4183,우주를 정복하는 딱 10가지 지식,베키 스메서스트 송근아 지웅배 천문학 자연과학 천문학 일반 사람 우주
4097,짧고 쉽게 쓴 시간의 역사,스티븐 호킹 레오나르드 믈로디노프 전대호 물리학 일반 천문학 자연과학 물리학 천문...
3992,NEWTON HIGHLIGHT 뉴턴 하이라이트 136 우주 대도감,뉴턴프레스 천문학 자연과학 천문학 일반 우주 우리 워드 소개 태양계 관측 은하 ...
3927,우주와 천체의 원리를 그림으로 쉽게 풀이한 천문학 사전,후타마세 도시후미 토쿠마루 유우 나카무라 도시히로 조민정 전영범 천문학 자연과학 ...
4107,"100개의 별, 우주를 말하다",플로리안 프라이슈테터 유영미 이희원 재미있는 우주이야기 과학 천문학 재미있는 과학...
4087,우리는 모두 별에서 왔다,윤성철 천문학 자연과학 조선일보 미디어 추천 천문학 일반
4033,비욘드 그래비티,매일경제 국민보고대회팀 과학 천문학 자연과학 과학 일반 천문학 일반 우주 세계
4010,아시모프의 코스모스,아이작 아시모프 이강환 재미있는 우주이야기 과학 천문학 재미있는 과학이야기 자연과...
4226,지구인의 우주공부,이명현 경향신문 재미있는 우주이야기 동아일보 과학 천문학 재미있는 과학이야기 자연...


In [93]:
similar_books_by_title('왜나는너를사랑하는가')

1319
[('왜 나는 너를 사랑하는가', 1319), ('불안', 1716), ('폭풍의 언덕', 1390), ('제인 에어 2', 1424), ('여행의 기술', 1968), ('제인 에어 1', 1405), ('위대한 개츠비', 1235), ('자기 앞의 생', 1181), ('낭만적 연애와 그 후의 일상', 1396), ('알랭 드 보통의 영혼의 미술관', 2710), ('사서함 110호의 우편물 ', 1221)]


In [102]:
similar_books_by_title('불안')

1716
10
[('오늘 내가 살아갈 이유', 1995), ('삶이 내게 말을 걸어올 때', 1854), ('타인의 시선을 의식해 힘든 나에게', 1840), ('걷기예찬', 2056), ('스물아홉 생일, 1년 후 죽기로 결심했다', 1776), ('내 인생의 해답', 1759), ('살아갈 날들을 위한 공부', 1879), ('술 취한 코끼리 길들이기', 1803), ('힘 빼기의 기술', 2107), ('제 인생에 답이 없어요', 1838)] [(1995, 0.27721530549635287), (1854, 0.21101851854094147), (1840, 0.1854225088048294), (2056, 0.17787730983252192), (1776, 0.16433700329860135), (1759, 0.1570125605905346), (1879, 0.15521988916872864), (1803, 0.14338643890172406), (2107, 0.13189644395739852), (1838, 0.13006440505892206)]


In [107]:
similar_books_by_title('코스모스')

3769
[('아빠, 천체관측 떠나요!', 4240), ('천문학자는 별을 보지 않는다', 1646), ('그림 속 별자리 신화', 4177), ('우리는 모두 별에서 왔다', 4087), ('100개의 별, 우주를 말하다', 4107), ('비욘드 그래비티', 4033), ('우주를 정복하는 딱 10가지 지식', 4183), ('NEWTON HIGHLIGHT 뉴턴 하이라이트 136 우주 대도감', 3992), ('평행우주', 4171), ('떨림과 울림', 3787)]


In [116]:
similar_books_by_title('셰익스')

1532
11
[('셰익스피어 5대 희극', 1532), ('초판본 셰익스피어 4대 비극', 1534), ('셰익스피어 4대 비극 컬렉션 세트', 1505), ('햄릿', 1356), ('파우스트 1', 1285), ('파우스트 2', 1348), ('베니스의 상인', 1506), ('[예스리커버] 햄릿 · 로미오와 줄리엣', 1379), ('돈키호테 1 ', 1590), ('신곡 세트', 1217), ('두 도시 이야기', 1490)] [1532, 1534, 1505, 1356, 1285, 1348, 1506, 1379, 1590, 1217]


In [80]:
similar_books_by_title('데미안')

1135
[('수레바퀴 아래서', 1213), ('단테의 신곡', 1404), ('시로 납치하다', 1587), ('싯다르타', 1186), ('초판본 데미안', 1168), ('눈먼 자들의 도시', 1509), ('충만한 힘', 1414), ('이방인', 1169), ('다섯째 아이', 1564), ('징비록', 2164)] [(1213, 0.23840285033511735), (1404, 0.13997406671202509), (1587, 0.11892128942481647), (1186, 0.11273742496297835), (1168, 0.10294567438308935), (1509, 0.09316788751361883), (1414, 0.09240065355744462), (1169, 0.09128865535302025), (1564, 0.08803003520733388), (2164, 0.07611463782965613)]


In [94]:
book_info_total.loc[[1213, 1404, 4062, 1587, 1186, 1168, 1509, 1414, 1169, 1564, 2164],['title', 'feature']]

,title,feature
1213,수레바퀴 아래서,헤르만 헤세 노벨문학상 수상작가 해외문학상 수상작 독일소설 노벨상수상자 노벨문...
1404,단테의 신곡,단테 양억관 서양 고전문학 고전문학 안사면0원사면영원 요즘책방 책읽어드립니다 ...
4062,그림으로 읽는 뇌과학의 모든 것,박문호 뇌과학 뇌과학 일반 자연과학 뇌가내를안다면 신경 내용 그림 강의 과학
1587,시로 납치하다,류시화 한국 시 시 노벨문학상 수상작가 해외문학상 수상작 희곡 일상을위로하다 ...
1186,싯다르타,헤르만 헤세 독일소설 북클러버의선택
1168,초판본 데미안,헤르만 헤세 이순학 독일소설 요즘책방 책읽어드립니다
1509,눈먼 자들의 도시,주제 사라마구 정영목 중남미소설 포르투갈 스페인 노벨문학상 수상작가 해외문학상 ...
1414,충만한 힘,파블로 네루다 정현종 세계의 시 시 노벨문학상 수상작가 해외문학상 수상작 희곡 ...
1169,이방인,알베르 까뮈 김화영 프랑스소설 노벨상수상자 방탄책방 방탄RM이추천한책 북클러...
1564,다섯째 아이,도리스 레싱 영미소설 영미 장편소설 노벨문학상 수상작가 해외문학상 수상작 노벨...


### 저자, 키워드, 카테고리 기반 책 추천
- 여러 단어를 한번에 검색하려면 띄어쓰기로 연결해도 되지만, 그러면 결과가 안나오기 쉬움
- 한 단어 검색을 권장

yes24 기반 키워드 (ex. 방송, 언론사 추천 등)

In [93]:
similar_books_by_keyword('책읽어드립니다')

64
[('넛지', 64), ('부의 미래, 누가 주도할 것인가', 62), ('앞으로 3년 경제전쟁의 미래', 178), ('7대 이슈로 보는 돈의 역사 2', 214), ('세상을 읽는 새로운 언어, 빅데이터', 231), ('스킨 인 더 게임 Skin in the Game', 949), ('사다리 걷어차기', 974), ('포지셔닝', 420), ('행동경제학', 147), ('인플레이션 이야기', 58)] [64, 62, 178, 214, 231, 949, 974, 420, 147, 58]


In [74]:
book_info_total.loc[[659, 569, 2567, 850, 1505, 1405, 1274, 1356, 1467, 1552, 1290], ['title', 'feature']]

,title,feature
659,타인의 고통,수전 손택 사회비평 사회학 국제사회비평 사회 정치 사회학일반 비판 역사의빛과그림...
569,"총, 균, 쇠",재레드 다이아몬드 김진준 사회비평 주제로 읽는 역사 사회학 국제사회비평 사회 정치...
2567,망고와 수류탄,기시 마사히코 정세경 사회비평 경향신문 주제로 읽는 역사 사회학 국제사회비평 사회...
850,국가는 왜 실패하는가,대런 애쓰모글루 제임스 A. 로빈슨 최완규 장경덕 사회비평 사회사상(사) 사회학 ...
1505,셰익스피어 4대 비극 컬렉션 세트,윌리엄 셰익스피어 고전문학 시나리오 시 서양 고전문학 희곡 요즘책방 책읽어드립...
1405,제인 에어 1,샬롯 브론테 영미소설 영미 장편소설 연애 사랑소설 테마소설 여성서사 영화드라마...
1274,사람은 무엇으로 사는가,레프 톨스토이 홍대화 러시아소설 서양 고전문학 고전문학 인생 변화 톨스 깨달
1356,햄릿,윌리엄 셰익스피어 최종철 고전문학 시나리오 시 서양 고전문학 희곡 비극 불멸의...
1467,홍계월전,유광수 홍선주 한국 고전문학 고전문학 여성
1552,구토,장 폴 사르트르 임호경 프랑스소설 서양 고전문학 고전문학 20세기고전 사르트르...


In [76]:
similar_books_by_keyword('동아일보')

4085
[('감염 도시', 4085), ('유전체, 다가온 미래 의학', 3991), ('아무도 죽지 않는 세상 ', 4235), ('초월', 4109), ('쉽게 쓴 후성유전학', 4160), ('두 얼굴의 백신', 3908), ('마음의 상처로 죽을 수도 있을까', 3897), ('잔혹한 진화론', 4017), ('우리가 몰랐던 바이러스 이야기', 3848), ('차이나는 클라스 의학·과학 편 ', 4041)] [(4085, 1.0000000000000002), (3991, 0.10374941508680026), (4235, 0.10360391045713291), (4109, 0.09265956754915144), (4160, 0.08289422090460109), (3908, 0.08078237647346191), (3897, 0.0711354464137919), (4017, 0.07062014892748165), (3848, 0.06916199980811692), (4041, 0.06774698471448222)]


카테고리, 장르

In [75]:
similar_books_by_keyword('시집')

1994
[('백석 평전', 1994), ('마음이 흐르는 대로', 1698), ('누울래? 일어날래? 괜찮아? 밥먹자', 2022), ('이토록 멋진 문장이라면', 1992), ('광고천재 이제석', 1967), ('고백', 1875), ('장일순 평전', 2642), ('필 때도 질 때도 동백꽃처럼', 1591), ('숨결이 바람 될 때', 1669), ('고양이를 버리다', 2136)] [(1994, 1.0), (1698, 0.08915180550757294), (2022, 0.08859104478130242), (1992, 0.08246748624142869), (1967, 0.06695623839270429), (1875, 0.06068547945467959), (2642, 0.05801700749463145), (1591, 0.05514937177098929), (1669, 0.05321444261296514), (2136, 0.05138691926314215)]


In [84]:
similar_books_by_keyword('자연과학')

4146
[('게놈 익스프레스', 4146), ('모든 순간의 물리학', 3789), ('종의 기원', 1100), ('숨결이 바람 될 때', 1669), ('설민석의 조선왕조실록', 2148), ('쇼코의 미소', 1198), ('반응하지 않는 연습', 3739), ('당신은 아무 일 없던 사람보다 강합니다', 3681), ('라플라스의 마녀', 1208), ('야밤의 공대생 만화', 3809)] [(4146, 0.9999999999999994), (3789, 0.12703854916025115), (1100, 0.12379722984424782), (1669, 0.11813659509080629), (2148, 0.1118100691633454), (1198, 0.10903451727952165), (3739, 0.10839462145239799), (3681, 0.10144419668449645), (1208, 0.08637236677958571), (3809, 0.08619439730275545)]


In [86]:
similar_books_by_keyword('휴먼 에세이')

1994
[('백석 평전', 1994), ('마음이 흐르는 대로', 1698), ('누울래? 일어날래? 괜찮아? 밥먹자', 2022), ('이토록 멋진 문장이라면', 1992), ('광고천재 이제석', 1967), ('고백', 1875), ('장일순 평전', 2642), ('필 때도 질 때도 동백꽃처럼', 1591), ('숨결이 바람 될 때', 1669), ('고양이를 버리다', 2136)] [(1994, 1.0), (1698, 0.08915180550757294), (2022, 0.08859104478130242), (1992, 0.08246748624142869), (1967, 0.06695623839270429), (1875, 0.06068547945467959), (2642, 0.05801700749463145), (1591, 0.05514937177098929), (1669, 0.05321444261296514), (2136, 0.05138691926314215)]


저자

In [101]:
similar_books_by_keyword('알랭 드 보통')

1968
[('여행의 기술', 1968), ('먼 북소리', 1904), ('혼자 여행을 다녀왔습니다', 1869), ('하늘 호수로 떠난 여행', 2072), ('나를 부르는 숲', 1692), ('그림으로 다시 떠난 여행', 2016), ('쉬운 천국', 1858), ('끌림', 1774), ('탐라는 제주', 2073), ('캠핑하루', 2027)] [1968, 1904, 1869, 2072, 1692, 2016, 1858, 1774, 2073, 2027]


In [92]:
book_info_total.loc[[1319, 1716, 1390, 1424, 1968, 1405, 1235, 1181, 1396, 2710, 1221], ['title', 'feature']]

,title,feature
1319,왜 나는 너를 사랑하는가,알랭 드 보통 정영목 프랑스소설 연애 사랑소설 테마소설 사랑의형태 봄날엔로맨스...
1716,불안,알랭 드 보통 정영목 외국 에세이 에세이 삶의 자세와 지혜 북클러버의선택 이정도...
1390,폭풍의 언덕,에밀리 브론테 영미소설 영미 장편소설 연애 사랑소설 테마소설 책읽아웃 소설
1424,제인 에어 2,샬롯 브론테 영미소설 영미 장편소설 연애 사랑소설 테마소설 여성서사 에어 제인
1968,여행의 기술,알랭 드 보통 정영목 여행 에세이 외국 에세이 여행에세이 여행 에세이 일상을위로...
1405,제인 에어 1,샬롯 브론테 영미소설 영미 장편소설 연애 사랑소설 테마소설 여성서사 영화드라마...
1235,위대한 개츠비,스콧 피츠제럴드 김욱동 영미소설 영미 장편소설 영화와 드라마 원작 연애 사랑소설...
1181,자기 앞의 생,로맹 가리 용경식 공쿠르상 수상작 프랑스소설 연애 사랑소설 해외문학상 수상작 테...
1396,낭만적 연애와 그 후의 일상,알랭 드 보통 동아일보 경향신문 YES24 올해의 책 중앙일보 2016년 올해의...
2710,알랭 드 보통의 영혼의 미술관,존 암스트롱 알랭 드 보통 김한영 교양 미술일반 예술 미술 예술사 예술일반 내손...


In [80]:
similar_books_by_keyword('나태주')

1106
[('꽃을 보듯 너를 본다', 1106), ('끝까지 남겨두는 그 마음', 1141), ('시가 나에게 살라고 한다', 1280), ('나태주, 시간의 쉼표', 1130), ('나태주, 시간의 쉼표 (소장판)', 1361), ('풀꽃', 1172), ('가장 예쁜 생각을 너에게 주고 싶다', 1160), ('너만 모르는 그리움', 1359), ('가지 말라는데 가고 싶은 길이 있다', 1133), ('그런 사람 또 없습니다', 1202), ('오래 보아야 예쁘다 너도 그렇다', 1366)]


In [82]:
similar_books_by_keyword('스티븐 호킹')

3873
[('그림으로 보는 시간의 역사', 3873), ('우주를 정복하는 딱 10가지 지식', 4183), ('짧고 쉽게 쓴 시간의 역사', 4097), ('NEWTON HIGHLIGHT 뉴턴 하이라이트 136 우주 대도감', 3992), ('우주와 천체의 원리를 그림으로 쉽게 풀이한 천문학 사전', 3927), ('100개의 별, 우주를 말하다', 4107), ('우리는 모두 별에서 왔다', 4087), ('비욘드 그래비티', 4033), ('아시모프의 코스모스', 4010), ('지구인의 우주공부', 4226), ('모든 사람을 위한 빅뱅 우주론 강의', 3965)]


In [86]:
similar_books_by_keyword('올리버 색스')

2944
[('아내를 모자로 착각한 남자', 2944), ('적정한 삶 ', 2875), ('행복의 정복', 3249), ('솔직하게, 상처 주지 않게', 3239), ('가족의 두 얼굴', 3020), ('그림으로 읽는 생생 심리학', 3259), ('사람이 힘겨운 당신을 위한 관계의 심리학', 3255), ('불안한 마음을 잠재우는 법', 2973), ('이게 다 심리학 덕분이야', 3141), ('인간 관계의 법칙', 3081), ('걱정이 많은 사람을 위한 심리학 수업', 2962)]


In [102]:
similar_books_by_keyword('올해의 책')

13
[('[예스리커버] 돈의 속성 ', 13), ('존리의 부자되기 습관', 61), ('죽은 자의 집 청소', 1632), ('부자언니 부자특강', 32), ('살고 싶다는 농담', 1683), ('마음챙김의 시', 1119), ('애쓰지 않고 편안하게', 1615), ('일의 기쁨과 슬픔', 1178), ('알로하, 나의 엄마들', 1173), ('부의 대이동', 50), ('돈의 심리학', 28)]


In [103]:
similar_books_by_keyword('원작')

581
[('노마드랜드', 581), ('엘리트 세습', 679), ('가난의 문법', 839), ('그들은 왜 나보다 덜 내는가', 923), ('나는 옐로에 화이트에 약간 블루', 640), ('핀란드 사람들은 왜 중고가게에 갈까?', 992), ('쌀 재난 국가', 690), ('인간 섬', 933), ('우리도 행복할 수 있을까', 834), ('아이들의 계급투쟁', 1032), ('작은 꿈을 위한 방은 없다', 811)]
